In [1]:
import LSTM_encoder as s2s
import PyTorch_enc_dec_w_attention as fn
import LSTM_plotting as s2s_plt
import LSTM_process as proc

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# import and prepare data
ocr = pd.read_csv('./pit_data/SP1.txt', index_col='timestamp_utc')
mask_cols = ['porePressure_63cm_kPa', 'porePressure_113cm_kPa', 'porePressure_244cm_kPa']
ocr[mask_cols] = ocr[mask_cols].mask(ocr[mask_cols] < -14)
ocr['date'] = ocr.index.values.astype(float)

# add in forecasted rainfall
# this will take a while
for i in range(2,38,2):
    ocr = fn.add_max_rainfall(ocr,i,0,noise=False)

# rearrange cols
ocr = ocr.loc[:,['date', 'RG_mm','2hr_total_rfall', '2hr_max_rfall', '4hr_total_rfall', '4hr_max_rfall',
       '6hr_total_rfall', '6hr_max_rfall', '8hr_total_rfall', '8hr_max_rfall',
       '10hr_total_rfall', '10hr_max_rfall', '12hr_total_rfall',
       '12hr_max_rfall', '14hr_total_rfall', '14hr_max_rfall',
       '16hr_total_rfall', '16hr_max_rfall', '18hr_total_rfall',
       '18hr_max_rfall', '20hr_total_rfall', '20hr_max_rfall',
       '22hr_total_rfall', '22hr_max_rfall', '24hr_total_rfall',
       '24hr_max_rfall', '26hr_total_rfall', '26hr_max_rfall',
       '28hr_total_rfall', '28hr_max_rfall', '30hr_total_rfall',
       '30hr_max_rfall', '32hr_total_rfall', '32hr_max_rfall',
       '34hr_total_rfall', '34hr_max_rfall', '36hr_total_rfall',
       '36hr_max_rfall', 'VWC_20cm', 'VWC_50cm', 'VWC_93cm',
       'porePressure_63cm_kPa', 'porePressure_113cm_kPa',
       'porePressure_244cm_kPa']]

# scale data
data = ocr
data_scaling = preprocessing.MinMaxScaler(feature_range=(0,1)).fit(data.iloc[:,:])
data_scaled = pd.DataFrame(data_scaling.transform(data.iloc[:,:]),index=ocr.index)
data_scaled_df = data_scaled.fillna(-1)

# finally, prepare features and targets
features, targets, target_indices = fn.lstm_prep(data_scaled_df.index.values, data_scaled_df.values, 3, 36, 36)

# we choose 3 here for # of targets, because we're wanting to predict pore pressure, which is the last 3 columns
# 36 is the width of our "moving window". 36 time entries. This is supposed to represent 36 hours.

# create bounds for the prediction intervals
intervals = np.zeros(len(target_indices))
for i in range(0,len(intervals),targets.shape[1]):
    intervals[i]=1

intervals[intervals==0] = np.nan

binary_indices = np.copy(target_indices)

for i in range(0,len(binary_indices),36):
    binary_indices[i]=np.datetime64("NaT")
    
# set -1s to nan to be ignored
targets[targets==-1]=np.nan

# split training/testing data based on water years 1&2
split = 0.7
train_features = features[:int((len(features)*split))]  # use 70% as training
test_features = features[int((len(features)*split)):]  # use 30% as validation

train_targets = targets[:int((len(features)*split))]  # use 70% as training
test_targets = targets[int((len(features)*split)):]  # use 30% as validation
test_indices = target_indices[-test_targets.shape[0]*test_targets.shape[1]:]

# convert windowed data from np.array to PyTorch tensor
X_train, Y_train, X_test, Y_test = proc.numpy_to_torch(train_features, train_targets, test_features, test_targets)

print("Preparations complete!")

Preparations complete!


In [5]:
# specify model parameters and train
model = s2s.lstm_seq2seq(input_size = train_features.shape[2], hidden_size = 36)
loss = model.train_model(X_train, Y_train, n_epochs = 50, target_len = 3, batch_size = 36, training_prediction = 'mixed_teacher_forcing', teacher_forcing_ratio = 0.6, learning_rate = 0.01, dynamic_tf = False)

# plot predictions on train/test data
s2s_plt.plot_train_test_results(model, train_features, train_targets, test_features, test_targets)

  0%|                                                    | 0/50 [00:00<?, ?it/s]


510 36 44


RuntimeError: input.size(-1) must be equal to input_size. Expected 44, got 3

In [ ]:
# specify model parameters and train
model = lstm_encoder_decoder.lstm_seq2seq(input_size = X_train.shape[2], hidden_size = 15)
loss = model.train_model(X_train, Y_train, n_epochs = 50, target_len = ow, batch_size = 5, training_prediction = 'mixed_teacher_forcing', teacher_forcing_ratio = 0.6, learning_rate = 0.01, dynamic_tf = False)

# plot predictions on train/test data
plotting.plot_train_test_results(model, Xtrain, Ytrain, Xtest, Ytest)

plt.close('all')